## Sentinel-2 vegetation mask based on NDVI and BSI

### Service Definition

In [1]:
service = dict([('title', 'Sentinel-2 vegetation mask based on NDVI and BSI'),
                ('abstract', 'Sentinel-2 vegetation mask based on NDVI and BSI'),
                ('id', 'ewf-s2-vegetation-mask')])

**Number of classes**

Number of modes, which will be used to generate class membership

In [2]:
n_classes = dict([('id', 'n_classes'),
                  ('value', '1=-1|0,2=0|0.1,3=0.1|0.2,4=0.2|1'),
                  ('title', 'Classes and ranges for NDVI'),
                  ('abstract', 'Classes and ranges for NDVI (class_1=min|max,class_2=min|max)'),
                  ('maxOccurs', '20')])

In [3]:
b_classes = dict([('id', 'b_classes'),
                  ('value', '1=-1|-0.2,2=-0.2|0.1,3=0.1|0.2,4=0.2|1'),
                  ('title', 'Classes and ranges for BSI'),
                  ('abstract', 'Classes and ranges for BSI (class_1=min|max,class_2=min|max)'),
                  ('maxOccurs', '20')])

In [4]:
ndvi_threshold = dict([('id', 'ndvi_threshold'),
                   ('value', '0.3'),
                   ('title', 'NDVI threshold for the mask expression'),
                   ('abstract', 'NDVI threshold for the mask expression'),
                   ('maxOccurs', '1')])

In [5]:
bsi_threshold = dict([('id', 'bsi_threshold'),
                   ('value', '0'),
                   ('title', 'BSI threshold'),
                   ('abstract', 'BSI threshold'),
                   ('maxOccurs', '1')])

In [6]:
aoi = dict([('id', 'aoi'),
              ('title', 'Area of interest'),
              ('abstract', 'Area of interest'),
              ('value', '-70.5659,-13.0922,-69.1411,-12.4567')])

In [7]:
username = dict([('id', 'username'),
              ('title', 'Ellip username'),
              ('abstract', 'Ellip username'),
              ('value', '')])

In [8]:
api_key = dict([('id', 'api_key'),
              ('title', 'Ellip API key for data pipeline'),
              ('abstract', 'Ellip API key for data pipeline'),
              ('value', '')])

### <a name="runtime">Runtime parameter definition

**Input reference**

This is the Sentinel-2 catalogue entry URLs

In [9]:
input_references = ['https://catalog.terradue.com/ard-s2-boa-reflectances/search?uid=6276B1444A67F9C3C38F330D8A09B3462FF1836A',
                    'https://catalog.terradue.com/ard-s2-boa-reflectances/search?uid=D2CC3A1F5F9E61D22CDF7F7513A1A35E6AA76A0D',
                    'https://catalog.terradue.com/ard-s2-boa-reflectances/search?uid=AB3807E0B5158644C1052951F109C717FC62D289',
                    'https://catalog.terradue.com/ard-s2-boa-reflectances/search?uid=39BE1E4541AFD99D538E1FE32694E84B3E58FD7E']

### Workflow

#### Import the packages required for processing the data

In [10]:
import os
import sys

sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())
from helpers import *
%load_ext autoreload
%autoreload 2


sys.path.append('/opt/OTB/lib/python')
sys.path.append('/opt/OTB/lib/libfftw3.so.3')
os.environ['OTB_APPLICATION_PATH'] = '/opt/OTB/lib/otb/applications'
os.environ['LD_LIBRARY_PATH'] = '/opt/OTB/lib'
os.environ['ITK_AUTOLOAD_PATH'] = '/opt/OTB/lib/otb/applications'
os.environ['GDAL_DATA'] = '/opt/anaconda/share/gdal/'
import otbApplication

In [11]:
product_metadata = get_product_metadata(input_references, 
                                        username['value'],
                                        api_key['value'])

In [12]:
product_metadata

,cc,enclosure,enddate,identifier,platform,self,startdate,track,wkt,geometry
0,17.598377,https://store.terradue.com/ard-s2-boa-reflecta...,2019-06-29 14:57:39.024,6276B1444A67F9C3C38F330D8A09B3462FF1836A,S2B,https://catalog.terradue.com/ard-s2-boa-reflec...,2019-06-29 14:57:39.024,39,"POLYGON((-70.8415341437528 -13.6554833185667,-...","POLYGON ((-70.8415341437528 -13.6554833185667,..."
1,0.309651,https://store.terradue.com/ard-s2-boa-reflecta...,2019-06-29 14:57:39.024,D2CC3A1F5F9E61D22CDF7F7513A1A35E6AA76A0D,S2B,https://catalog.terradue.com/ard-s2-boa-reflec...,2019-06-29 14:57:39.024,39,"POLYGON((-69.9181391041721 -12.7524314684136,-...","POLYGON ((-69.9181391041721 -12.7524314684136,..."
2,26.911520,https://store.terradue.com/ard-s2-boa-reflecta...,2019-06-29 14:57:39.024,AB3807E0B5158644C1052951F109C717FC62D289,S2B,https://catalog.terradue.com/ard-s2-boa-reflec...,2019-06-29 14:57:39.024,39,"POLYGON((-69.9212607569422 -13.6568683838856,-...","POLYGON ((-69.9212607569422 -13.6568683838856,..."
3,4.473067,https://store.terradue.com/ard-s2-boa-reflecta...,2019-06-29 14:57:39.024,39BE1E4541AFD99D538E1FE32694E84B3E58FD7E,S2B,https://catalog.terradue.com/ard-s2-boa-reflec...,2019-06-29 14:57:39.024,39,"POLYGON((-70.835297389016 -12.7511412207275,-6...",POLYGON ((-70.83529738901601 -12.7511412207275...


In [13]:
min_date = min(product_metadata['startdate'])
max_date = max(product_metadata['enddate'])

#### Area of interest

In [14]:
x_min, y_min, x_max, y_max = [float(c) for c in aoi['value'].split(',')]

In [15]:
x_min, y_min, x_max, y_max

(-70.5659, -13.0922, -69.1411, -12.4567)

#### Input vsi URLs

Use the Sentinel-2 ARD using GDAL Virtual File System

In [16]:
vsi_list = [get_vsi_url(input_reference, 
                        username['value'],
                        api_key['value']) for input_reference in input_references]

Create a GDAL virtual dataset with the VSI URLs

In [17]:
vrt_options = gdal.BuildVRTOptions()

vrt = 'my.vrt'

ds_vrt = gdal.BuildVRT(vrt, vsi_list, options=vrt_options)
ds_vrt.FlushCache()

In [18]:
src_ds = gdal.Open(get_vsi_url(input_references[0], 
                               username['value'],
                               api_key['value']))

ds_vrt = gdal.Open(vrt,  gdal.OF_UPDATE)

for band in range(ds_vrt.RasterCount):
    
    band += 1
    print band, src_ds.GetRasterBand(band).GetDescription()
    
    src_band = ds_vrt.GetRasterBand(band)
    src_band.SetDescription(src_ds.GetRasterBand(band).GetDescription())  
    
ds_vrt.FlushCache()

1 B01
2 B02
3 B03
4 B04
5 B05
6 B06
7 B07
8 B08
9 B8A
10 B09
11 B11
12 B12
13 SCL
14 MSK_CLDPRB
15 MSK_SNWPRB


Create a new VRT with the Area of Interest (easier to do with gdal.Translate)

In [19]:
clipped_vrt = 'clipped.vrt'

gdal.Translate(clipped_vrt,
               vrt,
               projWin=[x_min, y_max, x_max, y_min],
               projWinSRS='EPSG:4326',
               format='VRT')

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f4ac4d45150> >

#### Band expressions

In [20]:
band_2 = 'im1b2'
band_4 = 'im1b4'
band_8 = 'im1b8'
band_11 = 'im1b11'
scl = 'im1b13'

In [21]:
ndvi_expression = '({1}-{0})/({1}+{0})'.format(band_4, band_8)
bsi_expression = '(({3}/{4}+{1}/{4})-({2}/{4}+{0}/{4}))/(({3}/{4}+{1}/{4})+({2}/{4}+{0}/{4}))'.format(band_2, band_4, band_8, band_11, '10000')
invalid_expression = '{0} == 3 || {0} == 8 || {0} == 9'.format(scl) 


# if ndvi  >= 0.3 and bsi <= 0 then IS vegetation, otherwise no vegetation
mask_expression = '{0} ? 128 : {1} >= {2} && {3} <= {4} ? 1 : 0'.format(invalid_expression,
                                                                       ndvi_expression, 
                                                                        ndvi_threshold['value'],
                                                                       bsi_expression,
                                                                        bsi_threshold['value'])



In [22]:
ndvi_expression, bsi_expression, invalid_expression, mask_expression


('(im1b8-im1b4)/(im1b8+im1b4)',
 '((im1b11/10000+im1b4/10000)-(im1b8/10000+im1b2/10000))/((im1b11/10000+im1b4/10000)+(im1b8/10000+im1b2/10000))*10000',
 'im1b13 == 3 || im1b13 == 8 || im1b13 == 9',
 'im1b13 == 3 || im1b13 == 8 || im1b13 == 9 ? 128 : (im1b8-im1b4)/(im1b8+im1b4) >= 0.3 && ((im1b11/10000+im1b4/10000)-(im1b8/10000+im1b2/10000))/((im1b11/10000+im1b4/10000)+(im1b8/10000+im1b2/10000))*10000 <= 0 ? 1 : 0')

**Vegetation index classes**

In [23]:
ndvi_classes = dict((int(k.strip()), v.strip().replace('|', ',')) for k,v in  
                             (item.split('=') for item in n_classes['value'].split(',')))


bsi_classes = dict((int(k.strip()), v.strip().replace('|', ',')) for k,v in  
                             (item.split('=') for item in b_classes['value'].split(',')))

In [24]:
ndvi_classes, bsi_classes

({1: '-1,0', 2: '0,0.1', 3: '0.1,0.2', 4: '0.2,1'},
 {1: '-1,-0.2', 2: '-0.2,0.1', 3: '0.1,0.2', 4: '0.2,1'})

** All expressions **

In [25]:
expressions = []

invalid_data = 128

for index, vi_class in enumerate([ndvi_classes, bsi_classes]):
    
    f = '{} ? {}'.format(invalid_expression, invalid_data)

    for _class in vi_class.keys():

        if index == 0: vi_expression = ndvi_expression
        if index == 1: vi_expression = bsi_expression
            
        expression = '{0} >= {1} && {0} < {2} ? {3}'.format(vi_expression,
                                                                vi_class[_class].split(',')[0],
                                                                vi_class[_class].split(',')[1],
                                                                _class)
                   

        f = '{} : {} '.format(f, expression)

    f = '{}: {}'.format(f, 0)
    
    expressions.append(f)
    
expressions.append('{} ? {} : 0'.format(invalid_expression, invalid_data))

expressions.append(mask_expression)

In [26]:
expressions

['im1b13 == 3 || im1b13 == 8 || im1b13 == 9 ? 128 : (im1b8-im1b4)/(im1b8+im1b4) >= -1 && (im1b8-im1b4)/(im1b8+im1b4) < 0 ? 1  : (im1b8-im1b4)/(im1b8+im1b4) >= 0 && (im1b8-im1b4)/(im1b8+im1b4) < 0.1 ? 2  : (im1b8-im1b4)/(im1b8+im1b4) >= 0.1 && (im1b8-im1b4)/(im1b8+im1b4) < 0.2 ? 3  : (im1b8-im1b4)/(im1b8+im1b4) >= 0.2 && (im1b8-im1b4)/(im1b8+im1b4) < 1 ? 4 : 0',
 'im1b13 == 3 || im1b13 == 8 || im1b13 == 9 ? 128 : ((im1b11/10000+im1b4/10000)-(im1b8/10000+im1b2/10000))/((im1b11/10000+im1b4/10000)+(im1b8/10000+im1b2/10000))*10000 >= -1 && ((im1b11/10000+im1b4/10000)-(im1b8/10000+im1b2/10000))/((im1b11/10000+im1b4/10000)+(im1b8/10000+im1b2/10000))*10000 < -0.2 ? 1  : ((im1b11/10000+im1b4/10000)-(im1b8/10000+im1b2/10000))/((im1b11/10000+im1b4/10000)+(im1b8/10000+im1b2/10000))*10000 >= -0.2 && ((im1b11/10000+im1b4/10000)-(im1b8/10000+im1b2/10000))/((im1b11/10000+im1b4/10000)+(im1b8/10000+im1b2/10000))*10000 < 0.1 ? 2  : ((im1b11/10000+im1b4/10000)-(im1b8/10000+im1b2/10000))/((im1b11/10000+im1

Provide a meaningful output name for the result

In [27]:
date_format = '%Y%m%dT%H%m%S'

output_name = 'VEGETATION-MASK-{0}-{1}'.format(min_date.strftime(date_format), 
                                               max_date.strftime(date_format))

Apply the Orfeo Toolbox BandMathX operator

In [28]:
BandMathX = otbApplication.Registry.CreateApplication('BandMathX')

BandMathX.SetParameterStringList('il', [clipped_vrt])
BandMathX.SetParameterString('out', 'temp_{0}.tif'.format(output_name))
BandMathX.SetParameterString('exp', ';'.join(expressions))
BandMathX.SetParameterOutputImagePixelType('out', otbApplication.ImagePixelType_uint8)

BandMathX.ExecuteAndWriteOutput()

0

In [32]:
expressions

['im1b13 == 3 || im1b13 == 8 || im1b13 == 9 ? 128 : (im1b8-im1b4)/(im1b8+im1b4) >= -1 && (im1b8-im1b4)/(im1b8+im1b4) < 0 ? 1  : (im1b8-im1b4)/(im1b8+im1b4) >= 0 && (im1b8-im1b4)/(im1b8+im1b4) < 0.1 ? 2  : (im1b8-im1b4)/(im1b8+im1b4) >= 0.1 && (im1b8-im1b4)/(im1b8+im1b4) < 0.2 ? 3  : (im1b8-im1b4)/(im1b8+im1b4) >= 0.2 && (im1b8-im1b4)/(im1b8+im1b4) < 1 ? 4 : 0',
 'im1b13 == 3 || im1b13 == 8 || im1b13 == 9 ? 128 : ((im1b11/10000+im1b4/10000)-(im1b8/10000+im1b2/10000))/((im1b11/10000+im1b4/10000)+(im1b8/10000+im1b2/10000))*10000 >= -1 && ((im1b11/10000+im1b4/10000)-(im1b8/10000+im1b2/10000))/((im1b11/10000+im1b4/10000)+(im1b8/10000+im1b2/10000))*10000 < -0.2 ? 1  : ((im1b11/10000+im1b4/10000)-(im1b8/10000+im1b2/10000))/((im1b11/10000+im1b4/10000)+(im1b8/10000+im1b2/10000))*10000 >= -0.2 && ((im1b11/10000+im1b4/10000)-(im1b8/10000+im1b2/10000))/((im1b11/10000+im1b4/10000)+(im1b8/10000+im1b2/10000))*10000 < 0.1 ? 2  : ((im1b11/10000+im1b4/10000)-(im1b8/10000+im1b2/10000))/((im1b11/10000+im1

In [37]:
band_names = ['ndvi_class',
             'bsi_class',
             'cloud_mask',
             'vegetation_mask']

metadata = dict()
metadata['B02'] = 'im1b2'
metadata['B04'] = 'im1b4'
metadata['B08'] = 'im1b8'
metadata['B11'] = 'im1b11'
metadata['SCL'] = 'im1b13'


ds_temp = gdal.Open('temp_{0}.tif'.format(output_name),  gdal.OF_UPDATE)

for band_index in range(ds_temp.RasterCount):
    

    metadata['BAND_EXPRESSION'] = '({})'.format(expressions[band_index])
    #print band, ds_temp.GetRasterBand(band).GetDescription()
    
    src_band = ds_temp.GetRasterBand(band_index+1)
    src_band.SetMetadata(metadata)
    src_band.SetDescription(band_names[band_index])  
    
ds_temp.FlushCache()


Transform it to a Cloud Optimized GeoTIFF

In [38]:
cog('temp_{0}.tif'.format(output_name),
    '{0}.tif'.format(output_name))

Create the results metadata

In [39]:
for properties_file in ['result', output_name]:

    date_format = '%Y-%m-%dT%H:%m:%SZ'
    
    if properties_file == 'result':
        
        title = 'Reproducibility notebook used for generating {0}'.format(output_name)
   
    else: 
      
        title = 'Vegetation mask from {0} to {1}'.format(min_date.strftime(date_format),
                                                         max_date.strftime(date_format))
        
    with open(properties_file + '.properties', 'wb') as file:
        
        file.write('title={0}\n'.format(title))
        
        file.write('date={0}/{1}\n'.format(min_date.strftime(date_format),
                                           max_date.strftime(date_format)))
        
        file.write('geometry={0}'.format(get_image_wkt(output_name + '.tif')))

#### Clean-up

In [40]:
os.remove(clipped_vrt)
os.remove(vrt)

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.